**Dataset Information**: Dataset is extracted from the electric current drive signals. The drive contains intact as well as defective components. Therefore, dataset has 11 classes based on the condition of the components. Aim is, to predict the correct component condition based on the input variables using **Deep Learning** technique. Tools used: **Keras TensorFlow** 

**Dataset Rights**: This dataset has been taken from "University of California Irvine Machine Learning Repository" for the knowledge purpose and all the rights for this dataset are reserved by them. For more details like content of the dataset, owner of the dataset and reference research paper, please refer the following link: https://archive.ics.uci.edu/ml/datasets/Dataset+for+Sensorless+Drive+Diagnosis

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
#from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

Step 2. Import and print data

In [3]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df = pd.read_csv(r"N:\connectxactions\actions.csv",index_col=0)
# Print first 10 samples
print(df.head(10))

   1  2  3  4  5  6  7  8  9  10  ...  35  36  37  38  39  40  41  42  target  \
0  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   2   0   0   0       3   
1  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   1   0   0   0       3   
2  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   1   0   0   0       3   
3  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   2   0   0   0       3   
4  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   2   0   0   0       3   
5  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   1   0   0   0       3   
6  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   2   0   0   0       3   
7  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   0   1   0   0   0       3   
8  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   2   1   0   0   0       5   
9  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   1   2   0   0   0       5   

   winner  
0       2  
1       1  
2       2  
3       1  
4       2  
5       1  
6       1  
7       2  


In [4]:
df

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873445,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,2,2,1,0,1,2
1873446,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,2,1,1,0,2,2
1873447,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,2,1,1,0,2,2
1873448,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,2,2,1,0,3,2


Column indices 0 to 47 are input variables (total 48 columns). Column index 48 is target column that contains 11 different classes (1 column). 

Step 3. Data pre-processing

In [5]:
# Check missing values
print(df.isna().sum())

1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
31        0
32        0
33        0
34        0
35        0
36        0
37        0
38        0
39        0
40        0
41        0
42        0
target    0
winner    0
dtype: int64


No missing values. 

In [6]:
# Remove missing values IF AVAILABLE and print first 10 samples
# df = df.dropna()
# print(df.head(10))
# print(df.shape)


In [7]:
# Divide data into features X and target (Classes) Y
columns = [str(item) for item in range(1, 43)]
columns.append('winner')
X = df.loc[:,columns]
Y = df.loc[:,'target']
print(X.shape)
print(Y.shape)

(1873450, 43)
(1873450,)


In [8]:
# Statistical summary of the variables
print(X.describe())

                  1             2             3             4             5  \
count  1.873450e+06  1.873450e+06  1.873450e+06  1.873450e+06  1.873450e+06   
mean   1.889786e-01  1.885222e-01  2.270949e-01  3.715872e-01  2.193557e-01   
std    5.284426e-01  5.278856e-01  5.717689e-01  6.937112e-01  5.634502e-01   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
50%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
75%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
max    2.000000e+00  2.000000e+00  2.000000e+00  2.000000e+00  2.000000e+00   

                  6             7             8             9            10  \
count  1.873450e+06  1.873450e+06  1.873450e+06  1.873450e+06  1.873450e+06   
mean   1.677995e-01  1.335376e-01  2.906851e-01  3.056030e-01  3.585972e-01   
std    5.015070e-01  4.524715e-01  6.324378e-01  6.

Scale of all the variables is different. Therefore, feature scaling is important.    

In [9]:
# Check for class imbalance
print(df.groupby(Y).size())

target
0    240544
1    268526
2    286482
3    273671
4    290463
5    270031
6    243733
dtype: int64


Since all the classes have same sample size, there is no class imbalance. 

In [10]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

To train the Neural Network, single target column must be converted into one hot encoded fomat. For more details, visit this link: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [11]:
# Convert target Y to one hot encoded Y for Neural Network
Y = pd.get_dummies(Y)
# If target is in string form, use following code:
# First encode target values as integers from string
# Then perform one hot encoding
# encoder = LabelEncoder()
# encoder.fit(Y)
# Y = encoder.transform(Y)
# Y = np_utils.to_categorical(Y)

In [12]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X = X.values
Y = Y.values

Step 4. Define Neural Network Model 

Two hidden layers are defined with "Rectified Linear Unit" (relu) and 15 neurons each. Furthermore, this is a multi-class classification problem and there are total 11 target clsses, therefore "softmax" activation function and 11 neurons are used in the output layer. For hidden layers, the number of neurons should be in between the input data dimension and the output data dimension. In this case, the input data has 48 variable columns and output classes are 11. Therefore, the number of neurons for the hidden layer should be in between 11 and 48. You can try different values for the number of neurons as well as different number of hidden layers.  

In [13]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = Sequential()
    model.add(Dense(15, input_dim = 43, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(15, activation = 'relu'))
    model.add(Dense(7, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

Input dimension (input_dim) is 48, because the input variable columns are 48. It will change as per the dimension of the input variables.

**Note**: If you use only one hidden layer, then it will be the case of simple Neural Network problem. But, if you use more than one hidden layers for example 3, it will be considered as the deep learning problem.  

In [14]:
# Create Keras Classifier and use predefined baseline model
estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 100000, verbose = 1)
# Try different values for epoch and batch size

C:\Users\ryans\AppData\Local\Temp\ipykernel_42084\111872092.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = baseline_model, epochs = 100, batch_size = 100000, verbose = 1)


Step 5. Define cross-validation and train pre-defined model 

In [15]:
# KFold Cross Validation
kfold = KFold(n_splits = 5, shuffle = True, random_state = seed)
# Try different values of splits e.g., 10

In [16]:
# Object to describe the result
#results = cross_val_score(estimator, X, Y, cv = kfold)
# Result
#print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

From the result above, the accuracy is 94% and it can be improved by techniques like feature extraction, selection and feature engineering. 

In [17]:
estimator.fit(X,Y)

Epoch 1/100
19/19 [==============================] - 1s 7ms/step - loss: 1.9579 - accuracy: 0.1558
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 1.9415 - accuracy: 0.1664
Epoch 3/100
19/19 [==============================] - 0s 9ms/step - loss: 1.9291 - accuracy: 0.1837
Epoch 4/100
19/19 [==============================] - 0s 7ms/step - loss: 1.9146 - accuracy: 0.1995
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 1.8961 - accuracy: 0.2123
Epoch 6/100
19/19 [==============================] - 0s 8ms/step - loss: 1.8746 - accuracy: 0.2225
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 1.8526 - accuracy: 0.2323
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 1.8311 - accuracy: 0.2406
Epoch 9/100
19/19 [==============================] - 0s 7ms/step - loss: 1.8126 - accuracy: 0.2471
Epoch 10/100
19/19 [==============================] - 0s 7ms/step - loss: 1.7974 - accuracy: 0.2517
Epoch 11/

In [18]:
estimator.predict([X[3].tolist()])

1/1 [==============================] - 0s 67ms/step


array([3])

In [30]:
estimator.model.save('test')

INFO:tensorflow:Assets written to: test\assets


In [31]:
import keras
model2 = keras.models.load_model('test')

In [32]:
np.argmax(model2.predict([X[0].tolist()]))

1/1 [==============================] - 0s 48ms/step


3